In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [2]:
with open('../data/anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [7]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        #self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
        self.inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        self.targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

        # Build the LSTM cell
        #cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)
        def build_cell(lstm_size, keep_prob):
            lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
            return drop
        cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
        self.initial_state = cell.zero_state(batch_size, tf.float32)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        #self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        seq_output = tf.concat(outputs, axis=1)
        x = tf.reshape(seq_output, [-1, lstm_size])
        
        with tf.variable_scope('softmax'):
            softmax_w = tf.Variable(tf.truncated_normal([lstm_size, num_classes], stddev=0.1))
            softmax_b = tf.Variable(tf.zeros(num_classes))
        
        self.logits = tf.matmul(x, softmax_w) + softmax_b
        self.prediction = tf.nn.softmax(self.logits, name='predictions')
        
        # Loss and optimizer (with gradient clipping)
        #self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        y_one_hot = tf.one_hot(self.targets, num_classes)
        y_reshaped = tf.reshape(y_one_hot, self.logits.get_shape())
        self.loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=y_reshaped)
        self.loss = tf.reduce_mean(self.loss)
        
        #self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.optimizer = train_op.apply_gradients(zip(grads, tvars))

In [8]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

In [9]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.4202...  0.3931 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3360...  0.1560 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.9438...  0.1590 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 6.0447...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.1006...  0.1580 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.8226...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.7023...  0.1580 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.6441...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.5253...  0.1580 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.4419...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.3647...  0.1571 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.3401...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 1/20...  Training Step: 103...  Training loss: 3.0437...  0.1580 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 3.0365...  0.1580 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 3.0436...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.0257...  0.1571 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.0227...  0.1590 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.0145...  0.1590 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.0337...  0.1580 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 2.9886...  0.1580 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 2.9921...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 3.0109...  0.1590 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 2.9770...  0.1570 sec/batch
Epoch: 1/20...  Training Step: 114...  Training loss: 2.9613...  0.1570 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 203...  Training loss: 2.4288...  0.1590 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.4348...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.4324...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.4424...  0.1590 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.4492...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.4218...  0.1570 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.4132...  0.1560 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.4250...  0.1570 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.4085...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.4509...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 213...  Training loss: 2.4136...  0.1610 sec/batch
Epoch: 2/20...  Training Step: 214...  Training loss: 2.4085...  0.1570 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 303...  Training loss: 2.2001...  0.1560 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 2.2068...  0.1560 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 2.1941...  0.1590 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 2.2198...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 2.2143...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 2.1925...  0.1571 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 2.1932...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 2.2071...  0.1570 sec/batch
Epoch: 2/20...  Training Step: 311...  Training loss: 2.1832...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 312...  Training loss: 2.1867...  0.1570 sec/batch
Epoch: 2/20...  Training Step: 313...  Training loss: 2.1761...  0.1580 sec/batch
Epoch: 2/20...  Training Step: 314...  Training loss: 2.1479...  0.1580 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 403...  Training loss: 2.0587...  0.1590 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 2.0587...  0.1700 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 2.0940...  0.1580 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 2.0606...  0.1571 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 2.0387...  0.1580 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 2.0297...  0.1570 sec/batch
Epoch: 3/20...  Training Step: 409...  Training loss: 2.0574...  0.1570 sec/batch
Epoch: 3/20...  Training Step: 410...  Training loss: 2.0918...  0.1570 sec/batch
Epoch: 3/20...  Training Step: 411...  Training loss: 2.0554...  0.1570 sec/batch
Epoch: 3/20...  Training Step: 412...  Training loss: 2.0264...  0.1571 sec/batch
Epoch: 3/20...  Training Step: 413...  Training loss: 2.0401...  0.1590 sec/batch
Epoch: 3/20...  Training Step: 414...  Training loss: 2.0838...  0.1580 sec/batch
Epoch: 3/20...  

...

Epoch: 19/20...  Training Step: 3567...  Training loss: 1.2422...  0.1580 sec/batch
Epoch: 19/20...  Training Step: 3568...  Training loss: 1.2590...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3569...  Training loss: 1.2189...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3570...  Training loss: 1.1920...  0.1600 sec/batch
Epoch: 19/20...  Training Step: 3571...  Training loss: 1.2402...  0.1590 sec/batch
Epoch: 19/20...  Training Step: 3572...  Training loss: 1.2388...  0.1560 sec/batch
Epoch: 19/20...  Training Step: 3573...  Training loss: 1.2477...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3574...  Training loss: 1.2242...  0.1590 sec/batch
Epoch: 19/20...  Training Step: 3575...  Training loss: 1.2249...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3576...  Training loss: 1.2312...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3577...  Training loss: 1.2316...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3578...  Training loss: 1.2426...  0.1560 se

Epoch: 19/20...  Training Step: 3665...  Training loss: 1.2352...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3666...  Training loss: 1.2148...  0.1581 sec/batch
Epoch: 19/20...  Training Step: 3667...  Training loss: 1.2305...  0.1582 sec/batch
Epoch: 19/20...  Training Step: 3668...  Training loss: 1.2258...  0.1580 sec/batch
Epoch: 19/20...  Training Step: 3669...  Training loss: 1.2233...  0.1580 sec/batch
Epoch: 19/20...  Training Step: 3670...  Training loss: 1.2247...  0.1591 sec/batch
Epoch: 19/20...  Training Step: 3671...  Training loss: 1.2380...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3672...  Training loss: 1.2329...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3673...  Training loss: 1.2142...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3674...  Training loss: 1.2399...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3675...  Training loss: 1.2114...  0.1570 sec/batch
Epoch: 19/20...  Training Step: 3676...  Training loss: 1.2334...  0.1580 se

Epoch: 20/20...  Training Step: 3763...  Training loss: 1.3273...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3764...  Training loss: 1.2350...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3765...  Training loss: 1.2261...  0.1600 sec/batch
Epoch: 20/20...  Training Step: 3766...  Training loss: 1.2511...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3767...  Training loss: 1.2025...  0.1590 sec/batch
Epoch: 20/20...  Training Step: 3768...  Training loss: 1.1880...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3769...  Training loss: 1.2173...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3770...  Training loss: 1.2241...  0.1560 sec/batch
Epoch: 20/20...  Training Step: 3771...  Training loss: 1.2358...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3772...  Training loss: 1.2139...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3773...  Training loss: 1.2155...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3774...  Training loss: 1.2207...  0.1580 se

Epoch: 20/20...  Training Step: 3861...  Training loss: 1.1962...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3862...  Training loss: 1.1987...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3863...  Training loss: 1.2125...  0.1590 sec/batch
Epoch: 20/20...  Training Step: 3864...  Training loss: 1.2006...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3865...  Training loss: 1.2268...  0.1600 sec/batch
Epoch: 20/20...  Training Step: 3866...  Training loss: 1.2089...  0.1581 sec/batch
Epoch: 20/20...  Training Step: 3867...  Training loss: 1.2106...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3868...  Training loss: 1.2020...  0.1600 sec/batch
Epoch: 20/20...  Training Step: 3869...  Training loss: 1.2131...  0.1580 sec/batch
Epoch: 20/20...  Training Step: 3870...  Training loss: 1.2143...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3871...  Training loss: 1.2052...  0.1570 sec/batch
Epoch: 20/20...  Training Step: 3872...  Training loss: 1.2229...  0.1570 se

Epoch: 20/20...  Training Step: 3959...  Training loss: 1.2094...  0.1560 sec/batch
Epoch: 20/20...  Training Step: 3960...  Training loss: 1.2048...  0.1570 sec/batch


## testing

In [10]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [11]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [12]:
tf.train.latest_checkpoint('checkpoints')
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i3960_l512.ckpt
Farther.

"Anna will the work in the wagater and he pursuishly. It sought it, if
they've not seen it out."

"Oh, then, inded that," he siin silent.

"Why, these death! Well, that, it's only in the carrying it in the stail,"
 the position, so shouting herself, and was simpining tomernow and agond
torance.

"I should not be often anyone. A mind of a tron, I chanced howe it all in
aristoca!" said Anna, stating his bitter with which. "I teing this woman
is anxieticed in tomorrow is how they'll greater the sights, and I can
not live to me, I could not to go out, I'll consincurate them what it
here about them that I am," he said to Anna with in smill, said that she
would nut he said when he coule obviously at him, and askidiously apparant
from the ball to see he how all this teress it was not lef in her
same, and he did not, becaue Anna, when a good might said nothing, andwhere
was an old man, and that in the most secondre